In [213]:
#!pip install pandas
from openpyxl import load_workbook
from openpyxl import Workbook
import pandas as pd
import csv
from math import isnan

In [403]:
def weatherFileToDict(filename, dict_weather):
    book = load_workbook(filename)
    sheet = book.active
    rows = list(sheet.rows)[1:]
    # row = rows[0]
    # cols = row[2:]
    # print(cols[0].value)
    for row in rows:
        ymd = row[0].value
        year = ymd[:4]
        month = ymd[5:7]+'월'
        if month[0] == '0':
            month = month[1:]
        day = ymd[8:10]+'일'
        if day[0] == '0':
            day = day[1:]

        area = row[1].value

        if dict_weather.get(year) == None:
            dict_weather[year] = {}
        if dict_weather[year].get(area) == None:
            dict_weather[year][area] = {}
        if dict_weather[year][area].get(month) == None:
            dict_weather[year][area][month] = {}
        if dict_weather[year][area][month].get(day) == None:
            dict_weather[year][area][month][day] = []

        for col in row[2:]:
            dict_weather[year][area][month][day].append(col.value)

# for cells_tuple in sheet['A1':'J1']:
#     for cell n cells_tuple:
#         print(cell.value)




In [434]:
def matchFileToDict(filename, dict_match, dict_homeAway):
    file = pd.read_csv(filename)

    for value_list in file.values:
        if type(value_list[2]) == type(''):
            date = value_list[2].split('(')[0].split('.')

        if value_list[10] == '우천취소' or value_list[10] == '미세먼지취소' or value_list[-1] != '0,9':
            continue

        year = repr(value_list[1])

        
        month = date[0]+'월'
        if month[0] == '0':
            month = month[1:]
        day = date[1]+'일'
        if day[0] == '0':
            day = day[1:]

        time = value_list[3]
        field = value_list[9]

        str_list = value_list[4].split('v')
        str_list[1] = str_list[1][1:]
        if not str_list[0][-1].isdigit():
             continue
#        match = str_list[0] + ' ' + str_list[1]
        
        match = ''
        num = []
        for s in str_list:
            num_string = ''
            for ch in s:
                if ch.isdigit():
                    num_string += ch
                    continue
                match+=ch
            num.append(num_string)
            match+=' '
              
        if num[0] < num[1]:
            temp = num[0]
            num[0] = num[1]
            num[1] = temp
            match = match.split(' ')[1] + ' ' + match.split(' ')[0]
        
        homeAway = [0, 0] # 0: away, 1: home
        #이긴팀 홈?어웨이?
        for f in dict_homeAway[match.split(' ')[0]]:
            if f == field:
                homeAway[0] = 1
        #진팀 홈?어웨이?
        for f in dict_homeAway[match.split(' ')[1]]:
            if f == field:
                homeAway[1] = 1
            
        
        if dict_match.get(year)==None:
            dict_match[year] = {}
        if dict_match[year].get(month)==None:
            dict_match[year][month] = {}
        if dict_match[year][month].get(day)==None:
            dict_match[year][month][day] = {}
        if dict_match[year][month][day].get(field) == None:
            dict_match[year][month][day][field] = {}
        if dict_match[year][month][day][field].get(match) == None:
            dict_match[year][month][day][field][match] = {}
        if dict_match[year][month][day][field][match].get(time) == None:
            dict_match[year][month][day][field][match][time] = []
            
        dict_match[year][month][day][field][match][time].append(match.split(' ')[0])
        dict_match[year][month][day][field][match][time].append(num[0])
        dict_match[year][month][day][field][match][time].append(homeAway[0])
        dict_match[year][month][day][field][match][time].append(match.split(' ')[1])
        dict_match[year][month][day][field][match][time].append(num[1])
        dict_match[year][month][day][field][match][time].append(homeAway[1])

In [435]:
def mergeDicts(dict_match, dict_weather, dict_area):
    for year in dict_match.keys():
        for month in dict_match[year].keys():
            for day in dict_match[year][month].keys():
                for field in dict_match[year][month][day]:
                    for match in dict_match[year][month][day][field]:
                        for time in dict_match[year][month][day][field][match]:
                            if field == '고척':
                                for i in range(4):
                                    dict_match[year][month][day][field][match][time].append('-')
                                continue
                            area = dict_area[field]
                            list_weather_all = dict_weather[year][area][month][day]
                            dict_match[year][month][day][field][match][time].append(list_weather_all[0]) #평균기온
                            if list_weather_all[3] == '-':
                                list_weather_all[3] = '0.0'
                            dict_match[year][month][day][field][match][time].append(list_weather_all[3]) #강수량
                            dict_match[year][month][day][field][match][time].append(list_weather_all[4]) #평균풍속
                            dict_match[year][month][day][field][match][time].append(list_weather_all[5]) #상대습도

잠실, 고척, 수원, 사직, 광주, 문학, 창원, 대구, 대전, 울산, 마산, 포항, 청주  
서울, 서울, 수원, 부산, 광주, 인천, 창원, 대구, 대전, 울산, 서울, 포항, 청주  
  
KT | 한화   | LG| SK |키움 | 두산 | NC | KIA| 삼성     |롯데  
수원|대전,청주| 잠실|문학|고척 | 잠실 | 창원 | 광주| 대구,포항 |사직,울산

In [436]:
dict_area={'잠실':'서울', '고척':'서울', '수원':'수원', '사직':'부산', '광주':'광주', '문학':'인천', '창원':'창원', '대구':'대구', '대전':'대전', '울산':'울산', '마산':'서울', '포항':'포항', '청주':'청주'}
dict_homeAway={'KT':['수원'], '한화':['대전','청주'], 'LG':['잠실'], 'SK':['문학'], '키움':['고척'], '두산':['잠실'], 'NC':['창원','마산'], 'KIA':['광주'], '삼성':['대구','포항'], '롯데':['사직','울산'], '넥센':['고척'], '드림':[''], '나눔':['']}

In [437]:
dict_match = {}
filename = './match_data.csv'
matchFileToDict(filename, dict_match, dict_homeAway)
print(dict_match)

{'2018': {'3월': {'24일': {'마산': {'NC LG': {'14:00': ['NC', '4', 1, 'LG', '2', 0]}}, '잠실': {'삼성 두산 ': {'14:00': ['삼성', '6', 0, '두산', '3', 1]}}, '문학': {'SK 롯데': {'14:00': ['SK', '6', 1, '롯데', '5', 0]}}, '광주': {'KT KIA ': {'14:00': ['KT', '5', 0, 'KIA', '4', 1]}}, '고척': {'넥센 한화': {'14:00': ['넥센', '6', 1, '한화', '3', 0]}}}, '25일': {'마산': {'NC LG': {'14:00': ['NC', '7', 1, 'LG', '1', 0]}}, '잠실': {'두산 삼성': {'14:00': ['두산', '5', 1, '삼성', '4', 0]}}, '문학': {'SK 롯데': {'14:00': ['SK', '5', 1, '롯데', '0', 0]}}, '광주': {'KIA KT': {'14:00': ['KIA', '14', 1, 'KT', '1', 0]}}, '고척': {'한화 넥센 ': {'14:00': ['한화', '4', 0, '넥센', '1', 1]}}}, '27일': {'마산': {'NC 한화': {'18:30': ['NC', '9', 1, '한화', '6', 0]}}, '잠실': {'두산 롯데': {'18:30': ['두산', '5', 1, '롯데', '0', 0]}}, '문학': {'SK KT': {'18:30': ['SK', '8', 1, 'KT', '5', 0]}}, '광주': {'KIA 삼성': {'18:30': ['KIA', '17', 1, '삼성', '0', 0]}}, '고척': {'넥센 LG': {'18:30': ['넥센', '5', 1, 'LG', '4', 0]}}}, '28일': {'마산': {'한화 NC ': {'18:30': ['한화', '6', 0, 'NC', '2', 1]}}, '잠실': {'

In [438]:
dict_weather = {}
for year in range(3):
    year += 2018
    filename = './weather'+repr(year)+'.xlsx'
    weatherFileToDict(filename, dict_weather)
#print(dict_weather)

In [439]:
# dict_match = merge dict_match and dict_weather
mergeDicts(dict_match, dict_weather, dict_area)
print(dict_match)

{'2018': {'3월': {'24일': {'마산': {'NC LG': {'14:00': ['NC', '4', 1, 'LG', '2', 0, '7.6', '0.0', '7.6', '61.9']}}, '잠실': {'삼성 두산 ': {'14:00': ['삼성', '6', 0, '두산', '3', 1, '7.6', '0.0', '7.6', '61.9']}}, '문학': {'SK 롯데': {'14:00': ['SK', '6', 1, '롯데', '5', 0, '5.6', '0.0', '8.3', '80.9']}}, '광주': {'KT KIA ': {'14:00': ['KT', '5', 0, 'KIA', '4', 1, '9.8', '0.0', '4', '65.0']}}, '고척': {'넥센 한화': {'14:00': ['넥센', '6', 1, '한화', '3', 0, '-', '-', '-', '-']}}}, '25일': {'마산': {'NC LG': {'14:00': ['NC', '7', 1, 'LG', '1', 0, '9.6', '0.0', '6.1', '65.6']}}, '잠실': {'두산 삼성': {'14:00': ['두산', '5', 1, '삼성', '4', 0, '9.6', '0.0', '6.1', '65.6']}}, '문학': {'SK 롯데': {'14:00': ['SK', '5', 1, '롯데', '0', 0, '5.7', '0.0', '7.6', '89.5']}}, '광주': {'KIA KT': {'14:00': ['KIA', '14', 1, 'KT', '1', 0, '12.0', '0.0', '5', '75.1']}}, '고척': {'한화 넥센 ': {'14:00': ['한화', '4', 0, '넥센', '1', 1, '-', '-', '-', '-']}}}, '27일': {'마산': {'NC 한화': {'18:30': ['NC', '9', 1, '한화', '6', 0, '13.3', '0.0', '5.8', '61.1']}}, '잠실': {'두산 롯

In [440]:
book = Workbook()
sheet = book.active
sheet.title = 'merge'
sheet.append(['날짜','구장', '시간', '이긴팀', '이긴팀점수', '이긴팀홈어웨이', '진팀', '진팀점수', '진팀홈어웨이', '평균기온', '강수량', '평균풍속', '상대습도'])
for year in dict_match.keys():
    for month in dict_match[year].keys():
        for day in dict_match[year][month].keys():
            if len(month) == 2:
                m = '0' + month[0]
            else:
                m = month[0:2]
            
            if len(day) == 2:
                d = '0' + day[0]
            else:
                d = day[0:2]
            
            date = year + '-' + m + '-' + d
            for field in dict_match[year][month][day].keys():
                for match in dict_match[year][month][day][field]:
                    for time in dict_match[year][month][day][field][match]:
                        list_cur = [date, field, time]
                        for element in dict_match[year][month][day][field][match][time]:
                            list_cur.append(element)
                    print(list_cur)
                    sheet.append(list_cur)
book.save(filename=sheet.title+'.xlsx')

['2018-03-24', '마산', '14:00', 'NC', '4', 1, 'LG', '2', 0, '7.6', '0.0', '7.6', '61.9']
['2018-03-24', '잠실', '14:00', '삼성', '6', 0, '두산', '3', 1, '7.6', '0.0', '7.6', '61.9']
['2018-03-24', '문학', '14:00', 'SK', '6', 1, '롯데', '5', 0, '5.6', '0.0', '8.3', '80.9']
['2018-03-24', '광주', '14:00', 'KT', '5', 0, 'KIA', '4', 1, '9.8', '0.0', '4', '65.0']
['2018-03-24', '고척', '14:00', '넥센', '6', 1, '한화', '3', 0, '-', '-', '-', '-']
['2018-03-25', '마산', '14:00', 'NC', '7', 1, 'LG', '1', 0, '9.6', '0.0', '6.1', '65.6']
['2018-03-25', '잠실', '14:00', '두산', '5', 1, '삼성', '4', 0, '9.6', '0.0', '6.1', '65.6']
['2018-03-25', '문학', '14:00', 'SK', '5', 1, '롯데', '0', 0, '5.7', '0.0', '7.6', '89.5']
['2018-03-25', '광주', '14:00', 'KIA', '14', 1, 'KT', '1', 0, '12.0', '0.0', '5', '75.1']
['2018-03-25', '고척', '14:00', '한화', '4', 0, '넥센', '1', 1, '-', '-', '-', '-']
['2018-03-27', '마산', '18:30', 'NC', '9', 1, '한화', '6', 0, '13.3', '0.0', '5.8', '61.1']
['2018-03-27', '잠실', '18:30', '두산', '5', 1, '롯데', '0', 0, '1

['2019-06-27', '포항', '18:30', '두산', '9', 0, '삼성', '1', 1, '23.0', '26.1', '3.6', '95.4']
['2019-06-28', '잠실', '18:30', '두산', '3', 1, '롯데', '2', 0, '24.7', '0.0', '6.1', '67.3']
['2019-06-28', '대구', '18:30', '삼성', '9', 1, 'SK', '3', 0, '23.6', '7.0', '10.4', '86.5']
['2019-06-28', '수원', '18:30', 'KT', '9', 1, 'KIA', '1', 0, '24.0', '0.7', '5.4', '80.0']
['2019-06-28', '대전', '18:30', '키움', '6', 0, '한화', '5', 1, '25.4', '0.0', '5.4', '76.3']
['2019-06-28', '창원', '19:00', 'LG', '6', 0, 'NC', '2', 1, '22.4', '0.0', '6.8', '85.1']
['2019-06-29', '잠실', '17:00', '롯데', '4', 0, '두산', '0', 1, '23.1', '1.5', '4.3', '72.0']
['2019-06-29', '수원', '17:00', 'KT', '5', 1, 'KIA', '3', 0, '22.9', '3.9', '5', '85.6']
['2019-06-30', '잠실', '17:00', '롯데', '4', 0, '두산', '0', 1, '23.9', '0.0', '8.3', '73.5']
['2019-06-30', '대구', '17:00', '삼성', '7', 1, 'SK', '14', 0, '25.7', '0.0', '6.1', '74.8']
['2019-06-30', '창원', '17:00', 'NC', '9', 1, 'LG', '1', 0, '24.2', '0.0', '4.3', '79.8']
['2019-06-30', '수원', '17:00',

['2019-08-15', '사직', '18:00', '한화', '5', 0, '롯데', '11', 1, '27.4', '6.2', '11.5', '86.0']
['2019-08-16', '잠실', '18:30', '두산', '7', 1, 'LG', '4', 1, '25.9', '0.1', '7.9', '74.5']
['2019-08-16', '사직', '18:30', '한화', '2', 0, '롯데', '1', 1, '28.2', '0.0', '20.2', '76.0']
['2019-08-16', '수원', '18:30', 'KT', '9', 1, '삼성', '1', 0, '25.7', '1.0', '7.9', '75.5']
['2019-08-16', '광주', '18:30', 'KIA', '1', 1, 'SK', '0', 0, '26.5', '0.0', '6.5', '86.9']
['2019-08-16', '고척', '18:30', '키움', '6', 1, 'NC', '4', 0, '-', '-', '-', '-']
['2019-08-17', '잠실', '18:00', '두산', '9', 1, '롯데', '2', 0, '25.9', '0.0', '9', '69.5']
['2019-08-17', '대구', '18:00', '삼성', '7', 1, 'LG', '2', 0, '28.9', '0.0', '6.8', '62.5']
['2019-08-17', '창원', '18:00', 'NC', '9', 1, 'SK', '0', 0, '26.9', '0.0', '6.1', '72.0']
['2019-08-17', '광주', '18:00', 'KT', '4', 0, 'KIA', '3', 1, '26.9', '0.4', '8.6', '83.4']
['2019-08-17', '고척', '18:00', '한화', '8', 0, '키움', '4', 1, '-', '-', '-', '-']
['2019-08-18', '잠실', '17:00', '롯데', '3', 0, '두산',

In [441]:
print(dict_match)

{'2018': {'3월': {'24일': {'마산': {'NC LG': {'14:00': ['NC', '4', 1, 'LG', '2', 0, '7.6', '0.0', '7.6', '61.9']}}, '잠실': {'삼성 두산 ': {'14:00': ['삼성', '6', 0, '두산', '3', 1, '7.6', '0.0', '7.6', '61.9']}}, '문학': {'SK 롯데': {'14:00': ['SK', '6', 1, '롯데', '5', 0, '5.6', '0.0', '8.3', '80.9']}}, '광주': {'KT KIA ': {'14:00': ['KT', '5', 0, 'KIA', '4', 1, '9.8', '0.0', '4', '65.0']}}, '고척': {'넥센 한화': {'14:00': ['넥센', '6', 1, '한화', '3', 0, '-', '-', '-', '-']}}}, '25일': {'마산': {'NC LG': {'14:00': ['NC', '7', 1, 'LG', '1', 0, '9.6', '0.0', '6.1', '65.6']}}, '잠실': {'두산 삼성': {'14:00': ['두산', '5', 1, '삼성', '4', 0, '9.6', '0.0', '6.1', '65.6']}}, '문학': {'SK 롯데': {'14:00': ['SK', '5', 1, '롯데', '0', 0, '5.7', '0.0', '7.6', '89.5']}}, '광주': {'KIA KT': {'14:00': ['KIA', '14', 1, 'KT', '1', 0, '12.0', '0.0', '5', '75.1']}}, '고척': {'한화 넥센 ': {'14:00': ['한화', '4', 0, '넥센', '1', 1, '-', '-', '-', '-']}}}, '27일': {'마산': {'NC 한화': {'18:30': ['NC', '9', 1, '한화', '6', 0, '13.3', '0.0', '5.8', '61.1']}}, '잠실': {'두산 롯